In [ ]:
"""

Module: MODIS NPP Data Query and Export Script

Script Functionality:
This script queries MODIS Net Primary Productivity (NPP) data from Google Earth Engine (GEE) for a given region of interest (ROI) and time period. 
It processes the NPP data, visualizes it, and exports the results to Google Drive as GeoTIFF files.

Input Parameters:
A shapefile representing the region of interest (ROI), uploaded as an Earth Engine asset.
MODIS NPP data from the Earth Engine ImageCollection (MODIS/061/MOD17A3HGF).

Dependencies:
Earth Engine API : pip install earthengine-api (for accessing GEE data and functions)
Geemap : pip install geemap (for interacting with GEE in Python)


Version Information:
Version: v1.0
Change Log: October 08 2024.
"""

import ee
import geemap
# Initialize the Earth Engine API
ee.Authenticate()
ee.Initialize()

# Define MODIS NPP query function
def modis_npp(roi, start_time, end_time):
    start_time = ee.Date(start_time).millis()
    end_time = ee.Date(end_time).millis()

    dataset = ee.ImageCollection("MODIS/061/MOD17A3HGF") \
        .filter(ee.Filter.date(start_time, end_time)) \
        .filterBounds(roi) \
        .select('Npp') \
        .map(lambda image: image.clip(roi).multiply(0.0001))
    
    return dataset 

# Batch export function
def export_image_collection(img_col, scale, roi, task_name, file_name):
    # Get index list from the image collection
    index_list = img_col.reduceColumns(ee.Reducer.toList(), ["system:index"]).get("list").getInfo()

    # Extract geometry boundary of the ROI
    roi_geometry = roi.geometry().bounds().getInfo()['coordinates']  # Convert FeatureCollection to Geometry object

    for i in range(len(index_list)):
        image = img_col.filter(ee.Filter.eq("system:index", index_list[i])).first()
        task = ee.batch.Export.image.toDrive(
            image=image,
            description=f"{task_name}_{index_list[i]}",
            fileNamePrefix=f"{file_name}_{index_list[i]}",
            region=roi_geometry,  # Use valid GeoJSON boundary
            scale=scale,
            folder='GEE_Exports',
            crs="EPSG:4326",
            maxPixels=1e13
        )
        task.start()

# Batch display images (single band)
def batch_map(ic, vis_param, list_limit, pre_name):
    ic_list = ic.toList(list_limit)

    for i in range(ic_list.length().getInfo()):
        image = ee.Image(ic_list.get(i))
        image_date = image.get("system:index").getInfo()
        vis_layer = pre_name + "_" + image_date
        # Note: In Python, you can't directly use Map.addLayer, need to use GEE's UI interaction method. Code example below:
        print(f"Add Layer: {vis_layer}")

# Main function
def main():
    # Set region of interest (ROI)
    roi = ee.FeatureCollection('projects/ee-meiyang150269/assets/boundary')
    #roi = ee.FeatureCollection('users/your_username/your_table')
    
    # Query NPP data
    modis_npp_data = modis_npp(roi, "2021-01-01", "2022-12-31")
    
    # Display NPP data
    vis_param_npp = {'min': 0, 'max': 10000}
    batch_map(modis_npp_data, vis_param_npp, 100, "Modis_NPP")
    
    # Export data
    export_image_collection(modis_npp_data, 500, roi, "MODIS_NPP", "MODIS_NPP")

# Execute main function
main()

# End of the script

Add Layer: Modis_NPP_2021_01_01
Add Layer: Modis_NPP_2022_01_01


resample

In [ ]:
"""
Copyright (c) 2024 Yan Jin (jinyan@njupt.edu.cn) and Yong Ge (gey@lreis.ac.cn) 
Editor: Mei Yang (1024072120@njupt.edu.cn)
Date: October 08, 2024 
This code is protected by copyright and may not be copied or distributed without permission.

Module: LST Raster Resampling Script

Script Functionality:
This script resamples Net Primary Production (NPP) raster data based on a reference raster file. 
It reads raster files from a specified input folder, resamples them to match the resolution and projection 
of the reference raster, and saves the resampled output files to a designated folder.

Input Parameters:
A reference raster file (.tif) with the desired target resolution and projection.
A folder containing NPP raster files (.tif) that need to be resampled.

Dependencies:
GDAL : pip install gdal (for raster processing and reprojection)
Rasterio : pip install rasterio (optional, for additional raster operations)
NumPy : pip install numpy (for numerical array operations if needed)


Version Information:
Version: v1.0
Change Log: October 08 2024.
"""
from osgeo import gdal
import rasterio
from rasterio.enums import Resampling
import os
import numpy as np

referencefilepath = r'D:\ghana\0.005.tif'
inputfolderpath = r'D:\ghana\NPP\0.005'
inputfilepaths = []
outputfolderpath = r'D:\ghana\NPP\0.005\Resample'
outputfilepaths = []

# Get projection information
referencefile = gdal.Open(referencefilepath, gdal.GA_ReadOnly)
rfproj = referencefile.GetProjection()
bandreferencefile = referencefile.GetRasterBand(1)
rfTrans = referencefile.GetGeoTransform()
rfwidth = referencefile.RasterXSize
rfheight = referencefile.RasterYSize
nbands = referencefile.RasterCount

# Read data
file_list = os.listdir(inputfolderpath)
for filename in file_list:
    if filename.endswith('.tif'):
        inputpath = os.path.join(inputfolderpath, filename)
        outputpath = os.path.join(outputfolderpath, '0.005_' + filename)
        inputfilepaths.append(inputpath)
        outputfilepaths.append(outputpath)

print(inputfilepaths)

# Resampling
def ReprojectImages(inputFilePaths, outputFilePaths):
    for i in range(len(inputFilePaths)):
        inputfilePath = inputFilePaths[i]
        outputfilePath = outputFilePaths[i]
        # Get input raster file information
        inputrasfile = gdal.Open(inputfilePath, gdal.GA_ReadOnly)
        inputProj = inputrasfile.GetProjection()
        # Create resampled output file (set projection and six parameters)
        driver = gdal.GetDriverByName('GTiff')
        output = driver.Create(outputfilePath, rfwidth, rfheight, nbands, bandreferencefile.DataType)
        output.SetGeoTransform(rfTrans)
        output.SetProjection(rfproj)
        # Parameters: input dataset, output file, input projection, reference projection, resampling method (Nearest Neighbor, Bilinear, Cubic Convolution, etc.), callback function
        gdal.ReprojectImage(inputrasfile, output, inputProj, rfproj, gdal.GRA_NearestNeighbour)

ReprojectImages(inputfilepaths, outputfilepaths)

# End of the script

In [ ]:
"""
Copyright (c) 2024 Yan Jin (jinyan@njupt.edu.cn) and Yong Ge (gey@lreis.ac.cn)
Editor: Mei Yang (1024072120@njupt.edu.cn)
Date: November 03, 2024
This code is protected by copyright and may not be copied or distributed without permission.

Module: NPP Raster Resampling Script

Script Functionality:
This script resamples raster data files (.tif) located in a specified input folder. 
It processes each raster file to reduce its resolution by averaging values based on a defined scale factor (5), 
resulting in a new resolution of 0.05. The resampled raster files are saved in a designated output folder, 
with filenames updated to reflect the new resolution.

Input Parameters:
input_folder: A folder containing NPP raster files (.tif) to be resampled.
output_folder: A folder where the resampled NPP raster files will be saved.

Dependencies:
GDAL: pip install gdal (for raster processing and reprojection)
Rasterio: pip install rasterio (for raster file operations)
NumPy: pip install numpy (for numerical array operations)

Version Information:
Version: v1.0
Change Log: November 03, 2024.
"""

import os
import rasterio
from rasterio.enums import Resampling
import numpy as np

# Set input and output folder paths
input_folder = r"D:\ghana\11.3\NPP"  # Replace with your input folder path
output_folder = r"D:\ghana\11.3\NPP1"  # Replace with your output folder path

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Process all TIFF files in batch
for filename in os.listdir(input_folder):
    if filename.endswith('.tif'):
        input_file = os.path.join(input_folder, filename)  # Construct full path
        print(f"Processing file: {input_file}")
        
        with rasterio.open(input_file) as src:
            data = src.read(1)  # Read the first band
            profile = src.profile
            
            # Set new resolution (0.05) and calculate scale factor
            scale_factor = 5  # Aggregation factor from 0.01 to 0.05
            new_height = data.shape[0] // scale_factor
            new_width = data.shape[1] // scale_factor

            # Mean aggregation resampling
            resampled_data = data[:new_height * scale_factor, :new_width * scale_factor].reshape(
                new_height, scale_factor, new_width, scale_factor).mean(axis=(1, 3))

            # Update profile
            profile.update({
                "height": new_height,
                "width": new_width,
                "transform": src.transform * src.transform.scale(scale_factor, scale_factor)
            })

        # Generate new output filename
        new_filename = filename.replace("0.01", "0.05")  # Replace "0.01" in the filename with "0.05"
        output_file = os.path.join(output_folder, new_filename)

        # Export the result
        with rasterio.open(output_file, "w", **profile) as dst:
            dst.write(resampled_data, 1)

        print(f"File saved to: {output_file}")

print("All files have been processed.")

# End of the script

Processing file: D:\ghana\11.3\NPP\NPP_0.01_2021.tif
File saved to: D:\ghana\11.3\NPP1\NPP_0.05_2021.tif
Processing file: D:\ghana\11.3\NPP\NPP_0.01_2022.tif
File saved to: D:\ghana\11.3\NPP1\NPP_0.05_2022.tif
All files have been processed.
